<a href="https://colab.research.google.com/github/lucasvecchio/HardyL-Stalker-Clark-Kent-Project/blob/master/Portifolio_otm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 7.4 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=10a642e35b14dd7c7c2aebeacb5313013c57cd13e92a396d4eda12e55c436e51
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf  
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting

today = datetime.today().strftime('%Y-%m-%d')
assets = [i for i in input("escreva as ações separadas por vírgula: ").split(",")]
tipo = input("escolha: 1 - Max sharpe 2 - Min vol: ")
min_w = input("% mínima por ativo escolhido: ")
max_w = input("% máxima por ativo escolhido: ")
TLR = 0.02


len_assets = len(assets)
w1= 1/len_assets
weights = [w1]*len_assets
print(weights)
# Create a dataframe to store the adjusted close price of the stocks
df = pd.DataFrame()

# Store the adjusted close price of the sock into the df
for stock in assets:
  df1 = yf.download(assets,'2020-01-01')
  
df = df1['Adj Close']


returns = df.pct_change()
cov_matrix_annual = returns.cov()*252
port_variance = np.dot(weights,np.dot(cov_matrix_annual,weights))
port_volatility = np.sqrt(port_variance)
portfolio_simple_annual_return = np.sum(returns.mean()*weights)*252

# Sow the expected annual return, volatility (risk), and variance
percent_var = str(round(port_variance,2)*100)+'%'
percent_vola = str(round(port_volatility,2)*100)+'%'
percent_ret = str(round(portfolio_simple_annual_return,2)*100)+'%'

print("Dados Originais:")
print('Retorno anual esperado: '+percent_ret)
print('Risco/Vol anual: '+percent_vola)
print('Variancia anual: '+percent_var)
print("----------------------------------------------------")


#if(tipo=="1"):
# Portfolio Optimization

# Calculate the expected returns and the annualized sample covariance matrix of asset returns
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)


#for w in range(0,len_assets):
   # w = "w"+str([w])+"+"

if tipo == "1":
    # Optimize for maximum sharpe ratio
    ef = EfficientFrontier(mu,S,weight_bounds=(min_w,max_w))
    #ef.add_constraint(lambda w: w[0]+w[1]+w[2]+w[3] == 1)
    weights = ef.max_sharpe(risk_free_rate=TLR)
    cleaned_weights = ef.clean_weights() 
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    # Get the discrete allocation of each share per stock
    import cvxpy
    from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
    
    latest_prices = get_latest_prices(df)
    weights = cleaned_weights
    da = DiscreteAllocation(weights,latest_prices,total_portfolio_value = 6000)
    allocation,leftover = da.lp_portfolio()
    print("Dados Otimizados:")
    print('Discrete allocation: ',allocation)
    print('Funds remaining: ${:.2f}'.format(leftover))
    
else:
       # Optimize for maximum sharpe ratio
    ef = EfficientFrontier(mu,S,weight_bounds=(min_w,max_w))
    #ef.add_constraint(lambda w: w[0]+w[1]+w[2]+w[3] == 1)
    weights = ef.min_volatility()
    cleaned_weights = ef.clean_weights() 
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    # Get the discrete allocation of each share per stock
    import cvxpy
    from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
    
    latest_prices = get_latest_prices(df)
    weights = cleaned_weights
    da = DiscreteAllocation(weights,latest_prices,total_portfolio_value = 6400)
    allocation,leftover = da.lp_portfolio()
    print('Discrete allocation: ',allocation)
    print('Funds remaining: ${:.2f}'.format(leftover)) 

escreva as ações separadas por vírgula: aes,vug,bhp,sony,amd,qyld,mos,intc,v,mnst,aapl
escolha: 1 - Max sharpe 2 - Min vol: 1
% mínima por ativo escolhido: 0.02
% máxima por ativo escolhido: 0.3
[0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091]
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[****